In [27]:
import dxlib as dx
from dxlib.interfaces.external import yfinance
from dxlib import Date
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objs as go

In [47]:
class SignalStrategy(dx.Strategy):
    def __init__(self):
        super().__init__()
        self.field = "close"
        
    def execute(
            self, observation: any, history: dx.History, position: dx.Inventory=None,
    ) -> pd.Series:  # pd.Series[TradeSignal]
        levels = history.levels_unique()
        idx, _ = observation
        signals = pd.Series(
            dx.Signal(dx.Side.WAIT), index=pd.MultiIndex.from_tuples([idx], names=levels.keys())
        )
        
        security_level = history.schema.levels.index(dx.SchemaLevel.SECURITY)
        security = idx[security_level]

        # df = history.get_df({dx.SchemaLevel.SECURITY: [security]}, [self.field])
        r = np.random.rand(1)[0]
        if r > 0.9:
            signals.loc[idx] = dx.Signal(dx.Side.BUY , 1)
        elif r < 0.1:
            signals.loc[idx] = dx.Signal(dx.Side.SELL, 1)
            

        return signals

strategy = SignalStrategy()

In [48]:
securities = ["AAPL", "MSFT"]

In [49]:
api = yfinance.YFinanceAPI()
history = api.historical(securities, Date.prevdays(180), Date.today())
df = history.df.reset_index()
df = df.set_index("date")
print(df)
fig = px.line(df, y="close", color="security")
fig.show()

                          security        open        high         low  \
date                                                                     
2023-10-13 10:30:00  AAPL (equity)  181.419998  181.929993  178.139999   
2023-10-16 10:30:00  AAPL (equity)      176.75  179.080002  176.509995   
2023-10-17 10:30:00  AAPL (equity)  176.649994  178.419998  174.800003   
2023-10-18 10:30:00  AAPL (equity)  175.580002  177.580002  175.110001   
2023-10-19 10:30:00  AAPL (equity)  176.039993  177.839996  175.190002   
...                            ...         ...         ...         ...   
2024-04-04 10:30:00  MSFT (equity)   424.98999  428.670013  417.570007   
2024-04-05 10:30:00  MSFT (equity)   420.01001   426.51001  418.320007   
2024-04-08 10:30:00  MSFT (equity)  425.170013  427.279999  423.299988   
2024-04-09 10:30:00  MSFT (equity)  426.440002   427.73999  421.619995   
2024-04-10 15:56:00  MSFT (equity)  422.209991  424.029999  419.709991   

                          close    vo

In [50]:
schema = dx.Schema(
    levels=[dx.SchemaLevel.DATE, dx.SchemaLevel.SECURITY],
    fields=["close"],
    security_manager=dx.SecurityManager.from_list(["AAPL", "MSFT"]),
)

inventory = dx.Inventory({security: 0 for security in schema.security_manager.values()})
executor = dx.Executor(strategy, inventory)

In [53]:
signals = executor.run(history)
signals = signals.df
signals = signals.reset_index()
signals = signals.set_index("date")
print(signals)

                          security             signal
date                                                 
2023-10-13 10:30:00  AAPL (equity)  WAIT: None @ None
2023-10-16 10:30:00  AAPL (equity)  WAIT: None @ None
2023-10-17 10:30:00  AAPL (equity)  WAIT: None @ None
2023-10-18 10:30:00  AAPL (equity)  WAIT: None @ None
2023-10-19 10:30:00  AAPL (equity)  WAIT: None @ None
...                            ...                ...
2024-04-04 10:30:00  MSFT (equity)  WAIT: None @ None
2024-04-05 10:30:00  MSFT (equity)     SELL: 1 @ None
2024-04-08 10:30:00  MSFT (equity)  WAIT: None @ None
2024-04-09 10:30:00  MSFT (equity)  WAIT: None @ None
2024-04-10 15:56:00  MSFT (equity)  WAIT: None @ None

[246 rows x 2 columns]


In [59]:
# graph with signals
# if signal.side == dx.Buy, up arrow (green) else down arrow (red)
# if signal.side == dx.Wait, no arrow

fig = px.line(df, y="close", color="security")
for date, signal in signals.iterrows():
    security = signal["security"]
    signal = signal["signal"]
    
    # get same row by date in df
    row = df.loc[date]
    
    if signal.side == dx.Side.BUY:
        fig.add_trace(go.Scatter(x=[date], y=[row["close"]], mode="markers", marker=dict(color="green", symbol="triangle-up", size=10), showlegend=False))
    elif signal.side == dx.Side.SELL:
        fig.add_trace(go.Scatter(x=[date], y=[row["close"]], mode="markers", marker=dict(color="red", symbol="triangle-down", size=10), showlegend=False))

fig.show()